# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [7]:
import numpy as np
import pandas as pd
print("NumPy version:", np.__version__)
print("pandas version:", pd.__version__)

NumPy version: 2.0.2
pandas version: 2.2.3


In [8]:
tictactoe_df = pd.read_csv('tic-tac-toe.csv')

print(tictactoe_df.info())
print(tictactoe_df.describe())
print(tictactoe_df.columns)

print(tictactoe_df.isnull().sum())

column_list = tictactoe_df.columns.tolist()
print(column_list)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x    x    x    x    x    x    x    x  True
freq    418  378  418  378  458  378  418  378  418   626
Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')
TL       0
TM       0
TR       

In [9]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder

# making a copy for data integrity
tictactoe_copy_df = tictactoe_df.copy()

# X = input columns (features), columns 
X = tictactoe_copy_df.iloc[:, :-1] # all columns except the last, which contains 'class'

# y = output column (target)
# y = ['class']
y = tictactoe_copy_df['class']


# One-Hot Encode the input features
encoder = OneHotEncoder(sparse_output=False)  
X_encoded = encoder.fit_transform(X)

# Encode the target variable as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Normalize the encoded features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X_encoded)

# Display normalized data
print("Normalized Input Data:")
print(X_normalized)


Normalized Input Data:
[[0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 ...
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [10]:
# # Check if TensorFlow is installed and display its version
# try:
#     import tensorflow as tf
#     tensorflow_installed = f"TensorFlow is installed. Version: {tf.__version__}"
# except ImportError:
#     tensorflow_installed = "TensorFlow is not installed."

# tensorflow_installed


In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tabulate import tabulate


# Split the training and test data.
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Create a `Sequential` model
model = Sequential()

# Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. 
# Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.

model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))     # Input and first hidden layer
model.add(Dense(32, activation='relu'))                                 # Second hidden layer
model.add(Dense(16, activation='relu'))                                 # Third hidden layer
model.add(Dense(len(set(y)), activation='softmax'))                      # Output layer with Softmax


# Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Sparse categorical crossentropy for integer labels
              metrics=['accuracy']) # metrics, use `accuracy`


# Fit the training data.
model.fit(X_train, y_train, epochs=30, batch_size=8, validation_split=0.2)

# # Evaluate your neural network model with the test data.
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Prepare data for tabulate
evaluation_data = [
    ["Metric", "Value"],
    ["Test Accuracy", f"{test_accuracy:.2%}"],  # Convert to percentage format
    ["Test Loss", f"{test_loss:.4f}"]
]

# Print evaluation results as a table
print(tabulate(evaluation_data, headers="firstrow", tablefmt="grid"))



X_train shape: (766, 27)
y_train shape: (766,)
Epoch 1/30


C:\Users\Dan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4869 - loss: 0.6970 - val_accuracy: 0.7143 - val_loss: 0.5768
Epoch 2/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7013 - loss: 0.5763 - val_accuracy: 0.7532 - val_loss: 0.5154
Epoch 3/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7998 - loss: 0.4417 - val_accuracy: 0.7857 - val_loss: 0.4661
Epoch 4/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8490 - loss: 0.3683 - val_accuracy: 0.8117 - val_loss: 0.3886
Epoch 5/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8858 - loss: 0.2819 - val_accuracy: 0.8377 - val_loss: 0.3003
Epoch 6/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9332 - loss: 0.1777 - val_accuracy: 0.9221 - val_loss: 0.2365
Epoch 7/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9666 - loss: 0.1151 - val_accuracy: 0.9416 - val_loss: 0.1786
Epoch 8/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9877 - loss: 0.0703 - val_accuracy: 0.9545 - val_loss: 0.1351
Epo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [16]:
from tensorflow.keras.models import load_model
import numpy as np
from tabulate import tabulate

# Load the saved model
model = load_model('tic-tac-toe.keras') 

# Select a few random rows from the test set
random_indices = np.random.choice(len(X_test), size=5, replace=False)
X_sample = X_test[random_indices]
y_true = y_test[random_indices]

# Make predictions
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)  

# # Print results
# for i, index in enumerate(random_indices):
#     print(f"Sample {i+1}:")
#     print(f"Input Board: {X_sample[i]}")
#     print(f"True Label: {y_true[i]} (Encoded)")
#     print(f"Predicted Label: {y_pred[i]} (Encoded)")
#     print("---")

# Decode labels for readability
decoded_true = label_encoder.inverse_transform(y_true)
decoded_pred = label_encoder.inverse_transform(y_pred)

# Prepare data for tabulate
table_data = []
correct_count = 0
for i in range(len(random_indices)):
    correct = "✔️" if decoded_true[i] == decoded_pred[i] else "❌"
    if correct == "✔️":
        correct_count += 1
    table_data.append([i + 1, decoded_true[i], decoded_pred[i], correct])

# Print table
print(tabulate(
    table_data,
    headers=["Sample", "True Label", "Predicted Label", "Correct"],
    tablefmt="grid"
))

# Calculate percentage of correct predictions
percent_correct = (correct_count / len(random_indices)) * 100
print(f"\nPercent Correct: {percent_correct:.2f}%")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
+----------+--------------+-------------------+-----------+
|   Sample |   True Label |   Predicted Label | Correct   |
+==========+==============+===================+===========+
|        1 |            1 |                 1 | ✔️         |
+----------+--------------+-------------------+-----------+
|        2 |            1 |                 1 | ✔️         |
+----------+--------------+-------------------+-----------+
|        3 |            0 |                 0 | ✔️         |
+----------+--------------+-------------------+-----------+
|        4 |            1 |                 1 | ✔️         |
+----------+--------------+-------------------+-----------+
|        5 |            0 |                 0 | ✔️         |
+----------+--------------+-------------------+-----------+

Percent Correct: 100.00%


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [18]:
# Print evaluation results as a table
print(tabulate(evaluation_data, headers="firstrow", tablefmt="grid"))

+---------------+---------+
| Metric        | Value   |
+===============+=========+
| Test Accuracy | 96.88%  |
+---------------+---------+
| Test Loss     | 0.0972  |
+---------------+---------+


In [ ]:
# The model achieved low loss (<0.1) and high accuracy (>0.95)

**Which approach(es) did you find helpful to improve your model performance?**

In [22]:
# The model achieved the goals, no improvement required. 
# I will increase the number of epochs to 60 and make a custom Adam optimizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tabulate import tabulate
from tensorflow.keras.optimizers import Adam


# Split the training and test data.
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Create a `Sequential` model
model = Sequential()

# Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. 
# Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.

model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))     # Input and first hidden layer
model.add(Dense(32, activation='relu'))                                 # Second hidden layer
model.add(Dense(16, activation='relu'))                                 # Third hidden layer
model.add(Dense(len(set(y)), activation='softmax'))                      # Output layer with Softmax


# Create a custom Adam optimizer with a diferent learning rate
custom_adam = Adam(learning_rate=0.001)


# Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. 
model.compile(optimizer=custom_adam,
              loss='sparse_categorical_crossentropy',  # Sparse categorical crossentropy for integer labels
              metrics=['accuracy']) # metrics, use `accuracy`


# Fit the training data.
model.fit(X_train, y_train, epochs=60, batch_size=8, validation_split=0.2)

# # Evaluate your neural network model with the test data.
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Prepare data for tabulate
evaluation_data = [
    ["Metric", "Value"],
    ["Test Accuracy", f"{test_accuracy:.2%}"],  # Convert to percentage format
    ["Test Loss", f"{test_loss:.4f}"]
]

# Print evaluation results as a table
print(tabulate(evaluation_data, headers="firstrow", tablefmt="grid"))



X_train shape: (766, 27)
y_train shape: (766,)
Epoch 1/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6231 - loss: 0.6437 - val_accuracy: 0.6948 - val_loss: 0.5895
Epoch 2/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7067 - loss: 0.5551 - val_accuracy: 0.7468 - val_loss: 0.5301
Epoch 3/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7510 - loss: 0.4709 - val_accuracy: 0.7792 - val_loss: 0.4868
Epoch 4/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8151 - loss: 0.3980 - val_accuracy: 0.8377 - val_loss: 0.4123
Epoch 5/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9086 - loss: 0.2910 - val_accuracy: 0.8636 - val_loss: 0.3524
Epoch 6/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9440 - loss: 0.2070 - val_accuracy: 0.9156 - val_loss: 0.2421
Epoch 7/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9751 - loss: 0.1381 - val_accuracy: 0.9416 - val_loss: 0.1958
Epoch 8/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9866 - 

In [ ]:
# ...
# | Test Accuracy | 97.92%  |
# +---------------+---------+
# | Test Loss     | 0.0688  |
# +---------------+---------+

# Improvement of 1.04% in accuracy and 0.0284 in test loss. 